In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from urllib.parse import urlparse

In [2]:
url_base='https://www.vhcy.gov.tw'
url_work_table='/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

# Get page link
In this website the work table has some page, so the function should find page link for next step.

In [3]:
pages_link=[]
pages_link.append(url)
for o in soup.find("div",class_="pager").find_all("a"):
    if o.text == "下一頁" or o.text == "»":
        pass
    else:
        link = url_base+o.get('href')
        pages_link.append(link)
        print(o.text,link)

2 https://www.vhcy.gov.tw/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b&page=2
3 https://www.vhcy.gov.tw/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b&page=3
4 https://www.vhcy.gov.tw/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b&page=4
5 https://www.vhcy.gov.tw/PageView/RowView?WebMenuID=1c791b28-2968-49c9-8d5a-32dceca8ad1b&page=5


# Get work table

The `get_work_page` function is used for getting each work page

In [4]:
work_page = soup.find("tbody")

In [5]:
def get_work_page(page_url,page=None,page_link_part='?page='):
    global url
    if page != None:
        page_url = page_url+page_link_part+str(page)
    if url == None:
        page_url = url
        print(page_url)
    g=requests.get(page_url)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup

In [6]:
def get_work_detail(link):
    work_detail = get_work_page(link).find("div",class_="newContent").text
    if work_detail.replace("\n","") == "":
        return None
    else:
        return get_work_page(link).find("div",class_="newContent").text.replace("\r","").replace("\t","").split("\n")

In [7]:
def get_work_title(title):
    try:
        new_title = re.search("\B[科,短,社,契,牙,秘,醫,部,室](.*)[員,理,工,師,生]",title).group(0)
    except AttributeError:
        print("ERROR:",title)
    return new_title

In [8]:
def get_work_originazation(title,link):
    try:
        origination = re.search("((.*)[組,部,科,室,心])",title).group(1)
    except AttributeError:
        g = get_work_detail(link)
        if g == None:
            return "check page"
        try:
            origination = [gg for gg in g if "職稱" in gg][0].split("：")[-1]
        except AttributeError:
            ggg = [gg for gg in g if gg != '']
            origination = ggg[ggg.index("職稱")+1]
        except:
            origination = "check page"
    return origination

In [9]:
def get_work_deadtime(link):
    g = get_work_detail(link)
    if g == None:
        return "check page"
    try:
        ggg = [gg for gg in g if gg != '']
        try:
            dead = re.findall("\d+年\d+月\d+日|\d+.\d+.\d+",ggg[ggg.index("上網期間")+1])[-1]
        except:
            for gggg in ggg:
                
                if "上網期間" in gggg or "報名方式" in gggg or "報名人員請於" in gggg or "日前" in gggg or "報名日期" in gggg or "（含）前" in gggg:
                    try:
                        dead = re.findall("\d+年\d+月\d+日|\d+.\d+.\d+",[gg for gg in g if "上網期間" in gg or "報名方式" in gg or "報名日期" in gg or "（含）前" in gg][0].split("：")[-1])[-1]
                    except:
                        dead = "check page"
    except:
        dead = "check page"
    
    finally:
        if 'dead' not in locals():
            dead = "check page"
        return dead

In [10]:
def get_work_table(table_,work_table):
    for p_i, p_item in enumerate(table_.find_all("tr")):
        if p_item.find('a'):
            p_item_a = p_item.find('a')
            title = p_item_a.text
            if "錄取公告" in title or '考試公告' in title or "甄試結果公告" in title or "核定" in title:
                break
            new_title = get_work_title(title)
            link_s = url_base+p_item_a.get('href')
            if get_work_detail(link_s) == None:
                break
            origination = get_work_originazation(title,link_s)
            try:
                dead_line = get_work_deadtime(link_s)
            except:
                print("Error",p_item)
            print(p_i-1,new_title,link_s,dead_line, origination)
            work_table.append([new_title, dead_line, origination, link_s ])
    return work_table

In [11]:
work_table=[]
work_table = get_work_table(work_page,work_table)
for p_i, p_item in enumerate(pages_link):
    table_ = get_work_page(p_item)

    work_table = get_work_table(table_,work_table)

-1 部「 112年度美沙冬治療品質提升服務試辦計畫」專案助理 https://www.vhcy.gov.tw/PageView/RowViewDetail?WebRowsID=6e1c8821-1430-4fbd-a38c-d873072b4bb3&UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7 check page 精神部
0 契約護理師 https://www.vhcy.gov.tw/PageView/RowViewDetail?WebRowsID=a9e1064e-01d1-4228-8b7f-76560f603c8b&UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7 112年1月10日 社區健康營造中心
0 部「 112年度美沙冬治療品質提升服務試辦計畫」專案助理 https://www.vhcy.gov.tw/PageView/RowViewDetail?WebRowsID=6e1c8821-1430-4fbd-a38c-d873072b4bb3&UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7 check page 精神部
1 契約護理師 https://www.vhcy.gov.tw/PageView/RowViewDetail?WebRowsID=a9e1064e-01d1-4228-8b7f-76560f603c8b&UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7 112年1月10日 社區健康營造中心
0 室契約辦事員 https://www.vhcy.gov.tw/PageView/RowViewDetail?WebRowsID=69641825-d326-4b19-bb5a-1bf60d2c45ec&UnitID=fbb

In [122]:
work_table=pd.DataFrame(work_table, columns=['召聘職稱','期限' ,"職缺單位" ,'連結'])

print(work_table)

                     召聘職稱          期限                            職缺單位  \
0       醫院嘉義分院秘書室薦任第8職等專員  check page                  臺中榮民總醫院嘉義分院秘書室   
1                 室專案短期助理  111年05月19日                             秘書室   
2                     科助理   111年5月26日                              牙科   
3                室契約健保申報員  111年05月27日                           醫務企管室   
4                 室契約行政助理       111年5                           醫務企管室   
5                   室專案社工  111年05月26日                             社工室   
6               醫院師(二)級醫師    110年5月4日  臺中榮民總醫院師(二)級醫師兼科主任(重症醫學部重症加護內科   
7                  契約行政助理    111年5月2日                        社區健康營造中心   
8             社區日照中心照顧服務員   111.04.14                  灣橋分院附設義仁社區日照中心   
9                  室契約辦事員  111年05月31日                           社會工作室   
10      醫院嘉義分院秘書室薦任第8職等專員  check page                  臺中榮民總醫院嘉義分院秘書室   
11                室專案短期助理  111年05月19日                             秘書室   
12                    科助理   111年5月26日              

In [145]:
get_work_page('https://www.vhcy.gov.tw//PageView/RowViewDetail?WebRowsID=2296c3cc-ea34-4273-beac-603a99803d97&amp;UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&amp;CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7').find("div",class_="newContent").text

'\n\n'

In [139]:
type(s)

NoneType

In [142]:
isinstance(s,'NoneType')

TypeError: isinstance() arg 2 must be a type or tuple of types